In [ ]:
!pip install scann tensorflow optuna --upgrade

# ScaNN Demo with GloVe Dataset

In [24]:
import numpy as np
import h5py
import os
import requests
import tempfile
import time
import optuna
import scann

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download dataset

In [2]:
with tempfile.TemporaryDirectory() as tmp:
    response = requests.get("http://ann-benchmarks.com/glove-100-angular.hdf5")
    loc = os.path.join(tmp, "glove.hdf5")
    with open(loc, 'wb') as f:
        f.write(response.content)
    
    glove_h5py = h5py.File(loc, "r")

In [3]:
list(glove_h5py.keys())

['distances', 'neighbors', 'test', 'train']

In [4]:
dataset = glove_h5py['train']
queries = glove_h5py['test']
print(dataset.shape)
print(queries.shape)

(1183514, 100)
(10000, 100)


### Create ScaNN searcher

In [5]:
normalized_dataset = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]
# configure ScaNN as a tree - asymmetric hash hybrid with reordering
# anisotropic quantization as described in the paper; see README

# use scann.scann_ops.build() to instead create a TensorFlow-compatible searcher
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 43:11: text format contains deprecated field "min_cluster_size"
2022-07-01 12:36:06.137659: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 249797
2022-07-01 12:36:14.720796: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:88] PartitionerFactory ran in 8.583060288s.


In [18]:
# save index
!mkdir scann
searcher.serialize('scann/')

In [21]:
# load index
searcher = scann.scann_ops_pybind.load_searcher('scann/')

[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 38:5: text format contains deprecated field "min_cluster_size"


In [22]:
def compute_recall(neighbors, true_neighbors):
    total = 0
    for gt_row, row in zip(true_neighbors, neighbors):
        total += np.intersect1d(gt_row, row).shape[0]
    return total / true_neighbors.size

### ScaNN interface features

In [23]:
# this will search the top 100 of the 2000 leaves, and compute
# the exact dot products of the top 100 candidates from asymmetric
# hashing to get the final top 10 candidates.
start = time.time()
neighbors, distances = searcher.search_batched(queries)
end = time.time()

# we are given top 100 neighbors in the ground truth, so select top 10
print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :10]))
print("Time:", end - start)

Recall: 0.90015
Time: 4.384388208389282


In [8]:
# increasing the leaves to search increases recall at the cost of speed
start = time.time()
neighbors, distances = searcher.search_batched(queries, leaves_to_search=150)
end = time.time()

print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :10]))
print("Time:", end - start)

Recall: 0.92392
Time: 3.3855273723602295


In [9]:
# increasing reordering (the exact scoring of top AH candidates) has a similar effect.
start = time.time()
neighbors, distances = searcher.search_batched(queries, leaves_to_search=150, pre_reorder_num_neighbors=250)
end = time.time()

print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :10]))
print("Time:", end - start)

Recall: 0.93145
Time: 4.06306791305542


In [10]:
# we can also dynamically configure the number of neighbors returned
# currently returns 10 as configued in ScannBuilder()
neighbors, distances = searcher.search_batched(queries)
print(neighbors.shape, distances.shape)

# now returns 20
neighbors, distances = searcher.search_batched(queries, final_num_neighbors=20)
print(neighbors.shape, distances.shape)

(10000, 10) (10000, 10)
(10000, 20) (10000, 20)


In [11]:
# we have been exclusively calling batch search so far; the single-query call has the same API
start = time.time()
neighbors, distances = searcher.search(queries[0], final_num_neighbors=5)
end = time.time()

print(neighbors)
print(distances)
print("Latency (ms):", 1000*(end - start))

[ 97478 846101 671078 727732 544474]
[2.5518737 2.539792  2.5383418 2.5097368 2.4656374]
Latency (ms): 2.3496150970458984


In [25]:
def objective(trial):
    num_leaves = trial.suggest_int("num_leaves", 800, 1400, step=200)
    num_leaves_to_search = trial.suggest_int("num_leaves_to_search", 100, 500, step=200)
    reorder = trial.suggest_int("reorder", 50, 150, step=50)
    
    searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").tree(
        num_leaves=num_leaves, num_leaves_to_search=num_leaves_to_search, training_sample_size=len(normalized_dataset)).score_ah(
        2, anisotropic_quantization_threshold=0.2).reorder(reorder).build()
    
    neighbors, distances = searcher.search_batched(queries)
    
    return compute_recall(neighbors, glove_h5py['neighbors'][:, :10])


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print(f"Value: {trial.value}")
print("Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2022-07-01 20:24:35,470] A new study created in memory with name: no-name-e7521fb7-262d-4a79-ac39-7543f53215d6
[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 43:11: text format contains deprecated field "min_cluster_size"
2022-07-01 20:24:36.507987: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 1183514
2022-07-01 20:25:07.638937: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:88] PartitionerFactory ran in 31.130865035s.
[I 2022-07-01 20:25:37,742] Trial 0 finished with value: 0.91082 and parameters: {'num_leaves': 800, 'num_leaves_to_search': 100, 'reorder': 50}. Best is trial 0 with value: 0.91082.
[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 43:11: text format contains deprecated field "min_cluster_size"
2022-07-01 20:25:3

Number of finished trials:  10
Best trial:
Value: 0.9897
Params:
    num_leaves: 800
    num_leaves_to_search: 500
    reorder: 150


In [26]:
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").tree(
    num_leaves=800, num_leaves_to_search=500, training_sample_size=len(normalized_dataset)).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(150).build()

[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 43:11: text format contains deprecated field "min_cluster_size"
2022-07-01 20:39:27.198433: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 1183514
2022-07-01 20:39:57.768290: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:88] PartitionerFactory ran in 30.569770166s.


In [29]:
start = time.time()
neighbors, distances = searcher.search_batched(queries)
end = time.time()

# we are given top 100 neighbors in the ground truth, so select top 10
print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :10]))
print("Time:", end - start)

Recall: 0.9897
Time: 24.332216024398804


In [31]:
start = time.time()
neighbors, distances = searcher.search(queries[0])
end = time.time()
print("Time:", end - start)

Time: 0.004163503646850586
